In [1]:
!pip install --upgrade transformers sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 200.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.50.0
    Uninstalling transformers-4.50.0:
      Successfully uninstalled transformers-4.50.0

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import urllib.request

link = "https://huggingface.co/ngxson/demo_simple_rag_py/raw/main/cat-facts.txt"
dataset = []

for line in urllib.request.urlopen(link):
  dataset.append(line.decode('utf-8'))

print(f'Loaded {len(dataset)} entries')

Loaded 150 entries


In [2]:
from sentence_transformers import SentenceTransformer

EMBEDDING_MODEL = 'sentence-transformers/all-MiniLM-L6-v2'
model = SentenceTransformer(EMBEDDING_MODEL)

# Each element in the VECTOR_DB will be a tuple (chunk, embedding)
# The embedding is a list of floats, for example: [0.1, 0.04, -0.34, 0.21, ...]
VECTOR_DB = []

def add_chunk_to_database(chunk):
  embedding = model.encode(chunk, convert_to_tensor=True)
  VECTOR_DB.append((chunk, embedding))

for i, chunk in enumerate(dataset):
  add_chunk_to_database(chunk)

In [3]:
from sentence_transformers.util import semantic_search

input_query = "how long cats live"
query_embedding = model.encode(input_query, convert_to_tensor=True)

VECTOR_DB_ITEMS = list(map(lambda x: x[1], VECTOR_DB))
retrieved_knowledge = semantic_search(query_embedding, VECTOR_DB_ITEMS, top_k=5)


print('Retrieved knowledge:')
for r in retrieved_knowledge[0]:
  print(f' - (similarity: {r["score"]:.2f}) {VECTOR_DB[r["corpus_id"]][0]}')

instruction_prompt = f'''You are a helpful chatbot.
Use only the following pieces of context to answer the question. Don't make up any new information:
{''.join([f' - {VECTOR_DB[r["corpus_id"]][0]}' for r in retrieved_knowledge[0]])}
'''


Retrieved knowledge:
 - (similarity: 0.82) When well treated, a cat can live twenty or more years but the average life span of a domestic cat is 14 years.

 - (similarity: 0.74) The oldest cat on record was Crème Puff from Austin, Texas, who lived from 1967 to August 6, 2005, three days after her 38th birthday. A cat typically can live up to 20 years, which is equivalent to about 96 human years.

 - (similarity: 0.68) Neutering a cat extends its life span by two or three years.

 - (similarity: 0.63) On average, cats spend 2/3 of every day sleeping. That means a nine-year-old cat has been awake for only three years of its life.

 - (similarity: 0.58) Cats sleep 16 to 18 hours per day. When cats are asleep, they are still alert to incoming stimuli. If you poke the tail of a sleeping cat, it will respond accordingly.



In [4]:
import transformers

tokenizer = transformers.AutoTokenizer.from_pretrained("ibm-granite/granite-3.2-2b-instruct")
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

pipeline = transformers.pipeline("text-generation", model="ibm-granite/granite-3.2-2b-instruct", tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [5]:
messages = [
    {
        "role": "system",
        "content": instruction_prompt,
    },
    {
        "role": "user",
        "content": input_query,
    }
]

outputs = pipeline(messages, max_new_tokens=256)

In [6]:
from IPython.display import display, Markdown

output = ""
for turn in outputs:
    for item in turn["generated_text"]:
        output += f"# {item['role']}\n\n{item['content']}\n\n"

display(Markdown(output))

# system

You are a helpful chatbot.
Use only the following pieces of context to answer the question. Don't make up any new information:
 - When well treated, a cat can live twenty or more years but the average life span of a domestic cat is 14 years.
 - The oldest cat on record was Crème Puff from Austin, Texas, who lived from 1967 to August 6, 2005, three days after her 38th birthday. A cat typically can live up to 20 years, which is equivalent to about 96 human years.
 - Neutering a cat extends its life span by two or three years.
 - On average, cats spend 2/3 of every day sleeping. That means a nine-year-old cat has been awake for only three years of its life.
 - Cats sleep 16 to 18 hours per day. When cats are asleep, they are still alert to incoming stimuli. If you poke the tail of a sleeping cat, it will respond accordingly.



# user

how long cats live

# assistant

Cats can live for a significant number of years, typically around 14 years on average. However, with proper care, including neutering, they can live up to 20 years, which is roughly equivalent to 96 human years. The oldest cat on record, Crème Puff, lived to be 38 years and three days old.

